In [1]:
import cv2
import numpy as np
from skimage.transform import ProjectiveTransform

In [2]:
def find_keypoints_and_descriptors(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def match_keypoints(des1, des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    return good_matches

def find_homography(kp1, kp2, matches):
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return homography

def warp_image(image, homography, output_size):
    warped_image = cv2.warpPerspective(image, homography, output_size)
    return warped_image

def blend_images(image1, image2):
    blended_image = cv2.addWeighted(image1, 0.5, image2, 0.5, 0)
    return blended_image

def load_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"No se pudo cargar la imagen en {path}.")
        else:
            images.append(img)
    return images


In [3]:

# Cargar imágenes
image_paths = ['pictures/S1.jpg', 'pictures/S2.jpg', 'pictures/S3.jpg', 'pictures/S4.jpg', 'pictures/S5.jpg']  # Ruta de las imágenes
images = load_images(image_paths)


In [4]:

if len(images) < 2:
    print("Se necesitan al menos dos imágenes para crear una panorámica.")
else:
    # Encontrar homografías entre imágenes consecutivas
    keypoints = []
    descriptors = []
    homographies = []

    for i in range(len(images) - 1):
        keypoints1, descriptors1 = find_keypoints_and_descriptors(images[i])
        keypoints2, descriptors2 = find_keypoints_and_descriptors(images[i + 1])
        if keypoints1 is None or keypoints2 is None or descriptors1 is None or descriptors2 is None:
            print(f"No se pudieron encontrar puntos clave y descriptores para las imágenes {i} y {i + 1}.")
        else:
            matches = match_keypoints(descriptors1, descriptors2)
            if len(matches) < 4:
                print(f"No se encontraron suficientes coincidencias entre las imágenes {i} y {i + 1}.")
            else:
                homography = find_homography(keypoints1, keypoints2, matches)
                if homography is None:
                    print(f"No se pudo calcular la homografía entre las imágenes {i} y {i + 1}.")
                else:
                    homographies.append(homography)
                    keypoints.append(keypoints1)
                    descriptors.append(descriptors1)

    if len(homographies) < len(images) - 1:
        print("No se pudo encontrar la homografía para todas las imágenes consecutivas.")
    else:
        # Aplicar warping para alinear todas las imágenes
        output_size = (images[0].shape[1], images[0].shape[0])
        warped_images = [images[0]]
        for i in range(1, len(images)):
            warped_image = warp_image(images[i], np.linalg.inv(homographies[i - 1]), output_size)
            warped_images.append(warped_image)

        # Mezclar imágenes alineadas
        blended_image = warped_images[0]
        for i in range(1, len(warped_images)):
            blended_image = blend_images(blended_image, warped_images[i])

        # Mostrar la imagen panorámica resultante
        cv2.imshow('Panoramic Image', blended_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()